At the onset, we need to import required libraries. Tweepy is a library that has been used here to fetch tweets from the website. Pandas is useful for printing the final result in tabular form. Json library is used for handling json data.

In [0]:
import tweepy
import json
import pandas as pd

Next, we store the different keys and tokens which is needed for accessing tweets from the website. All the keys have been replaced by * for security reasons.

In [0]:
consumer_key = "**"
consumer_secret = "**"
access_token = "**"
access_secret = "**"

The following block sets all required keys. Then, a call to the API is made which will handle the tweets.

In [0]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
twitter_api = tweepy.API(auth)

This next segment fetches all tweet data.  The variable tweets is used to iterate through the status objects of the cursor. Each status object's json data is stored in "._json" attribute. The json data in each status object is dumped as string in the file "twitter_data.jsonl". Additionaly a newline is also appended after each json data dump.



In [0]:
with open('twitter_data.jsonl', 'w') as dumpdata:
  for tweets in tweepy.Cursor(twitter_api.user_timeline, screen_name='@midasIIITD', tweet_mode="extended").items():
      json.dump(tweets._json, dumpdata)
      dumpdata.write('\n')


Moving on to the next part of the code, each line of the previously created file is read and stored in a list of strings.

In [0]:

with open('twitter_data.jsonl', 'r') as infile:
    content = infile.readlines()                    # content is a list of string storing each line of the file


A function is created which fetches the number of images in a tweet. Unlike other data, image may be stored in two location depending on the type of the tweet. If the tweet is actually a retweet, the image data is stored inside a list named "*media*" which is present inside a dictionary whose key is named "*entities*" which again is present inside a dictionary whose key is named "*retweeted_status*". In case the tweet is not a retweet the image is stored in list *"media"* which is stored inside a dictionary whose key is *"entities"*.

The function *get_imagecount()* takes the list *media* as a parameter and checks whether any of the list entry (which again are dictionaries) has "type" key value "photo". If yes, it is a image, if no, then it is not.



In [0]:
def get_imagecount(medialist=[]):
  count = 0
  for diction in medialist:
    if diction["type"] == "photo":
      count = count+1
  return count

The required data is fetched from the json dictionaries and stored in corresponding lists as strings.
Here too, if the tweet is a retweet, favorite count and retweet count is present in a dictionary with key "retweeted_status". If it is just a simple tweet, they are present in the main dictionary itself

In [0]:
textlist = []                             # list to store text of tweet
fav_count = []                            # list to store number of likes on tweet
retweet_count = []                        # list to store number of retweets
datelist = []                             # list to store date and time of tweet
image_count = []                          # list to store number of images in a tweet
for line in content:
  jsondata = json.loads(line)                       # converting from string to type dictionary
  isRetweet = False                                 # to check whether a tweet is a retweet or not 
  textlist.append(jsondata["full_text"])            
  try:
    fav_count.append(jsondata["retweeted_status"]["favorite_count"])                  # if the tweet is a retweet, favorite and retweet count is stored in a 
    retweet_count.append(jsondata["retweeted_status"]["retweet_count"])               # dictionary whose key is "retweeted_status". In case the tweet is a retweet,
    isRetweet = True                                                                  # try block gets successfully executed
  except:
    fav_count.append(jsondata["favorite_count"])                                      # in case the tweet is not a retweet, the try block causes an error and execution
    retweet_count.append(jsondata["retweet_count"])                                   # moves into except block
    isRetweet = False
  datelist.append(jsondata["created_at"])
  try :
    if isRetweet == False:                                                            # As explained in a previous markdown cell, this try block attempts to get 
      if get_imagecount(jsondata["entities"]["media"]) == 0 :                         # number of images based on whether a tweet is a retweet or not
         image_count.append("None")
      else :
         image_count.append(get_imagecount(jsondata["entities"]["media"]))
    else:
      if get_imagecount(jsondata["retweeted_status"]["entities"]["media"]) == 0 :     # if there is no media, "None" is appended
         image_count.append("None")
      else :
         image_count.append(get_imagecount(jsondata["retweeted_status"]["entities"]["media"]))
  except :
    image_count.append("None")                                                        # if the key "media" is absent, it suggests 0 images and "None" is appended

All the lists are put inside a dictionary with appropriate key names for conversion to a dataframe of pandas.

In [8]:
table_data = {'Text': textlist, 'Likes' : fav_count, 'Retweets' : retweet_count, 'Images': image_count, 'Date/Time': datelist}
print(len(textlist))
print(len(fav_count))
print(len(image_count))
print(len(datelist))
print(len(retweet_count))

344
344
344
344
344


The dictionary is converted into a dataframe so that it can be displayed in a tabular format.

In [9]:
df = pd.DataFrame(table_data)
print(df)

                          Date/Time Images  Likes  Retweets  \
0    Wed Apr 10 09:01:29 +0000 2019   None      0         0   
1    Wed Apr 10 04:51:26 +0000 2019      1      3         1   
2    Tue Apr 09 16:45:07 +0000 2019      1     59        14   
3    Tue Apr 09 05:04:27 +0000 2019   None     96        35   
4    Tue Apr 09 05:04:11 +0000 2019      1     37        17   
5    Mon Apr 08 19:38:09 +0000 2019   None     20        15   
6    Mon Apr 08 07:08:12 +0000 2019      1     18         2   
7    Mon Apr 08 03:27:42 +0000 2019      1      5         0   
8    Sun Apr 07 14:17:29 +0000 2019   None      0         0   
9    Sun Apr 07 14:17:09 +0000 2019   None      0         0   
10   Sun Apr 07 11:43:24 +0000 2019   None      1         1   
11   Sun Apr 07 06:55:19 +0000 2019   None      5         2   
12   Sun Apr 07 06:53:38 +0000 2019   None      4         1   
13   Sun Apr 07 05:32:27 +0000 2019   None      6         1   
14   Sun Apr 07 05:29:40 +0000 2019   None      7      

For easy viewing and modification, the dataframe can be saved into a **.csv** too.

In [0]:
df.to_csv("mined_data.csv")